In [ ]:
!pip install datasets

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Import libraries

In [ ]:
import os
import io
import warnings
from itertools import islice
import numpy as np
import torch
import torchvision.transforms as T
from datasets import load_dataset
from PIL import Image, ImageFile
import requests
from tqdm.auto import tqdm
from torchvision.transforms.functional import resize
import concurrent.futures
from urllib.parse import urlparse
from transformers import CLIPProcessor, CLIPModel

# Ignore PIL warnings
ImageFile.LOAD_TRUNCATED_IMAGES = True
warnings.filterwarnings("ignore", category=UserWarning)

## Load dataset in streaming mode with basic config

In [ ]:
dataset = load_dataset("laion/laion400m", split="train", streaming=True)

Resolving data files:   0%|          | 0/128 [00:00<?, ?it/s]

## Define preprocessing parameters


In [ ]:
IMAGE_SIZE = 256  # Target size for images
MAX_SAMPLES = 1000
BATCH_SIZE = 32
MAX_WORKERS = 8  # Number of parallel workers
TIMEOUT = 3
EXPECTED_VALID_RATIO = 0.6  # Expected ratio of valid images (helps with progress bars)

# Load CLIP model for feature extraction
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

##  Define the transforms pipeline

In [ ]:
transform = T.Compose([
    T.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

## Define helper functions for downloading and processing images

In [ ]:
def fetch_image(url, timeout=TIMEOUT):
    """Fetch image from URL with timeout."""
    try:
        response = requests.get(url, timeout=timeout, stream=True)
        response.raise_for_status()
        img = Image.open(io.BytesIO(response.content)).convert('RGB')
        return img
    except Exception as e:
        return None

def preprocess_item(item):
    """Process a single data item, downloading and transforming the image with the trnsformation pipeline."""
    try:
        # Check if the item is potentially unsafe like NSFW or invalid
        if item['NSFW'] in ['LIKELY', 'NSFW']:
            return None

        # Extract domain for analysis
        domain = urlparse(item['url']).netloc

        # Download the image
        img = fetch_image(item['url'])
        if img is None:
            return None

        # Apply transformations
        img_tensor = transform(img)

        # get CLIP features
        with torch.no_grad():
            clip_inputs = clip_processor(
                text=[item['caption']],
                images=img,
                return_tensors="pt",
                padding=True
            )
            clip_outputs = clip_model(**clip_inputs)
            image_features = clip_outputs.image_embeds[0]
            text_features = clip_outputs.text_embeds[0]

        # Create processed item
        processed_item = {
            'id': item['key'],
            'image': img_tensor,
            'caption': item['caption'],
            'image_features': image_features.cpu(),
            'text_features': text_features.cpu(),
            'width': item['original_width'],
            'height': item['original_height'],
            'aspect_ratio': item['original_width'] / item['original_height'] if item['original_height'] > 0 else 0,
            'domain': domain,
            'similarity': item['similarity']
        }
        return processed_item
    except Exception as e:
        return None

##  Main processing function

In [ ]:
def process_dataset(dataset, max_samples, batch_size=BATCH_SIZE, max_workers=MAX_WORKERS):
    """Process the dataset in batches with parallel workers."""
    small_dataset = list(islice(dataset, max_samples))
    print(f"Loaded {len(small_dataset)} samples from LAION 400m")

    # For analysis
    source_domains = {}
    processed_samples = []
    valid_count = 0
    aspect_ratios = []

    # Process in batches
    for i in range(0, len(small_dataset), batch_size):
        batch = small_dataset[i:i+batch_size]

        # Process batch in parallel
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            results = list(tqdm(
                executor.map(preprocess_item, batch),
                total=len(batch),
                desc=f"Processing batch {i//batch_size + 1}/{len(small_dataset)//batch_size + 1}"
            ))

        # Filter out None results (the failed processing ones)
        valid_results = [r for r in results if r is not None]

        # Update statistics
        valid_count += len(valid_results)
        for item in valid_results:
            domain = item['domain']
            if domain in source_domains:
                source_domains[domain] += 1
            else:
                source_domains[domain] = 1

            aspect_ratios.append(item['aspect_ratio'])
            processed_samples.append(item)

    # Calculate and display dataset statistics
    print(f"\nProcessing complete: {valid_count}/{len(small_dataset)} valid samples ({valid_count/len(small_dataset)*100:.2f}%)")

    # Calculate aspect ratio distribution
    aspect_ratios = np.array(aspect_ratios)
    print(f"Aspect ratio stats: mean={aspect_ratios.mean():.2f}, std={aspect_ratios.std():.2f}")

    # Show top domains
    top_domains = sorted(source_domains.items(), key=lambda x: x[1], reverse=True)[:10]
    print("\nTop 10 domains:")
    for domain, count in top_domains:
        print(f"  {domain}: {count} images ({count/valid_count*100:.2f}%)")

    return processed_samples


## Create data loaders for training


In [ ]:
def create_dataloaders(processed_samples, batch_size=32, train_ratio=0.9):
    """Create PyTorch DataLoader objects for training and validation."""
    class LAIONDataset(torch.utils.data.Dataset):
        def __init__(self, samples):
            self.samples = samples

        def __len__(self):
            return len(self.samples)

        def __getitem__(self, idx):
            item = self.samples[idx]
            return {
                'id': item['id'],
                'image': item['image'],
                'caption': item['caption'],
                'image_features': item['image_features'],
                'text_features': item['text_features']
            }

    # Split into train/val
    train_size = int(len(processed_samples) * train_ratio)
    train_samples = processed_samples[:train_size]
    val_samples = processed_samples[train_size:]

    train_dataset = LAIONDataset(train_samples)
    val_dataset = LAIONDataset(val_samples)

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )

    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )

    return train_loader, val_loader

In [ ]:
if __name__ == "__main__":
    # Process the dataset
    processed_samples = process_dataset(dataset, MAX_SAMPLES)

    # Create DataLoaders
    train_loader, val_loader = create_dataloaders(processed_samples)

    # Show example of a processed sample
    if len(processed_samples) > 0:
        example = processed_samples[0]
        print("\nExample processed sample:")
        print(f"ID: {example['id']}")
        print(f"Caption: {example['caption']}")
        print(f"Image tensor shape: {example['image'].shape}")
        print(f"Image features shape: {example['image_features'].shape}")
        print(f"Text features shape: {example['text_features'].shape}")

    print("\nPreprocessing pipeline complete!")

Loaded 1000 samples from LAION 400m


Processing batch 1/32:   0%|          | 0/32 [00:00<?, ?it/s]

Processing batch 2/32:   0%|          | 0/32 [00:00<?, ?it/s]

Processing batch 3/32:   0%|          | 0/32 [00:00<?, ?it/s]

Processing batch 4/32:   0%|          | 0/32 [00:00<?, ?it/s]

...

Processing batch 29/32:   0%|          | 0/32 [00:00<?, ?it/s]

Processing batch 30/32:   0%|          | 0/32 [00:00<?, ?it/s]

Processing batch 31/32:   0%|          | 0/32 [00:00<?, ?it/s]

Processing batch 32/32:   0%|          | 0/8 [00:00<?, ?it/s]


Processing complete: 698/1000 valid samples (69.80%)
Aspect ratio stats: mean=1.17, std=0.45

Top 10 domains:
  cdn.shopify.com: 50 images (7.16%)
  thumbs.dreamstime.com: 16 images (2.29%)
  rlv.zcache.com: 15 images (2.15%)
  i.pinimg.com: 15 images (2.15%)
  i.ytimg.com: 12 images (1.72%)
  i0.wp.com: 12 images (1.72%)
  images.fineartamerica.com: 9 images (1.29%)
  photos.smugmug.com: 9 images (1.29%)
  images-na.ssl-images-amazon.com: 9 images (1.29%)
  ecx.images-amazon.com: 9 images (1.29%)

Example processed sample:
ID: 9048
Caption: bedroom minimalist home interior storage for kids bedroom design
Image tensor shape: torch.Size([3, 256, 256])
Image features shape: torch.Size([512])
Text features shape: torch.Size([512])

Preprocessing pipeline complete!
